# In-depth: Comparison of Spline Mathods
In previous studies, our group used the `CSAPS` function from MATLAB, which implements a cubic smoothing spline based on an algorithm by Carl de Boor ([A Practical Guide to Splines, 1978](https://www.springer.com/de/book/9780387953663)).

The first `bletl_analysis` port of this method for growth rate calculation used `scipy.interpolate.UnivariateSpline` (`us`), which uses a different algorithm with subtle, but important differences.  With version `bletl_analysis >= 1.0.0`, we've switched to the `UnivariateCubicSmoothingSpline` (`ucss`) from the [`csaps`](https://github.com/espdev/csaps) package. It implement a Python port of the same spline algorithm used in MATLAB.

Both algorithms have their advantages/disadvantages. This notebook aims to show where they differ and how the automatic selection of smoothing factors works.

In [1]:
import ipywidgets
import numpy
import pandas
import pathlib
import logging
import matplotlib.cm
from matplotlib import pyplot
import scipy
import scipy.stats
import csaps

import bletl
import bletl_pro
import bletl_analysis

logging.basicConfig(level=logging.DEBUG) # format='%(levelname)s:%(message)s', 
logging.getLogger('matplotlib').setLevel(logging.WARNING)

### Parse the raw data file

In [2]:
filepath = pathlib.Path(r'..\bletl_analysis\tests\data\107-AR_Coryne-AR-2019-04-15-12-39-30.csv')
bldata = bletl.parse(filepath, lot_number=1515, temp=30)

### Plot the data
Now we can visualize the us and ucss. The first subplot shows the splines and the datapoints. The second one shows the derivatives.

If filterset 'BS3' is selected the function `get_mue` from bletl_analysis is used to calculate specific growth rates shown in the third subplot.


In [3]:
def plot_well(well:str, filterset:str, last_cycle:int=None):
    t, y = bldata[filterset].get_timeseries(well)
    tmax = t[-1]
    cmax = len(t)
    t = t[:last_cycle]
    y = y[:last_cycle]

    # get splines
    spline_us = bletl_analysis.get_crossvalidated_spline(t, y, method="us", k_folds=10)
    spline_ucss = bletl_analysis.get_crossvalidated_spline(t, y, method="ucss", k_folds=10)
    
    # derivative functions
    der_us = spline_us.derivative(1)
    der_ucss = spline_ucss.derivative(1)
    
    # plotting
    if filterset == 'BS3':
        fig, (up, middle, down) = pyplot.subplots(dpi=140, nrows=3, sharex='col', figsize=(12,14))
    else:
        fig, (up, middle) = pyplot.subplots(dpi=140, nrows=2, sharex='col', figsize=(12,14/3*2))
        
    # highly resolved time values
    x = numpy.linspace(0, t[-1], 1000)
        
    up.scatter(t, y, marker='x', label='data')
    up.plot(x, spline_us(x), label='us', color='orange')
    up.plot(x, spline_ucss(x), label='ucss', color='blue')
    up.set_ylabel(filterset)
    
    middle.axhline(0, color='gray', linewidth=1, linestyle=':')
    middle.plot(x, der_us(x), label='ucss derivative', color='orange')
    middle.plot(x, der_ucss(x), label='us derivative', color='blue')
    middle.set_ylabel('derivative')
    
    # calculates get_mue only for filterset BS20
    if filterset == 'BS3':    
        # us
        mue_us = bletl_analysis.get_mue(bldata[filterset], wells=[well], method='us', last_cycle=last_cycle)
        df_mue_us = mue_us.get_unified_dataframe()
        df_mue_us.reset_index(inplace=True)
        us_value = numpy.array(df_mue_us[well], dtype=pandas.Series)
        
        # ucss
        mue_ucss = bletl_analysis.get_mue(bldata[filterset], wells=[well], method='ucss', last_cycle=last_cycle)
        df_mue_ucss = mue_ucss.get_unified_dataframe()
        df_mue_ucss.reset_index(inplace=True)
        ucss_value = numpy.array(df_mue_ucss[well], dtype=pandas.Series)
        
        # plot
        down.axhline(0, color='gray', linewidth=1, linestyle=':')
        time = numpy.array(df_mue_us["time in h"], dtype=pandas.Series)
        down.plot(time, us_value, label='US derivative', color='orange')
        down.plot(time, ucss_value, label='UCSS derivative', color='blue')
        down.set_xlabel('time   [h]')
        down.set_ylabel('specific growth rate   [1/h]')
        down.set_ylim(-0.5, 1)
    else:
        middle.set_xlabel('time   [h]')
    
    up.legend()
    # add second x-axis showing cycle numbers at the top
    x2 = up.twiny()
    x2.set_xlabel('cycle number   [-]')
    x2.set_xlim(0, cmax)
    up.set_xlim(0, tmax)
    pyplot.show()
    return

ipywidgets.interact(
    plot_well,
    well=list(bldata['pH'].time.columns),
    filterset=list(bldata.keys()),
    last_cycle=range(10, len(bldata['pH'].time)+1)
);

interactive(children=(Dropdown(description='well', options=('A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', '…

The interactive plot above shows that the `UnivariateCubicSmoothingSpline`  (ucss) has generally more wiggles in the approximation.

The `UnivariateSpline` however often has extreme derivates at the end of the curve. For example when `filterset=BS3, last_cycle=64` (a few cycles after a DO-peak), the `us` method has a strong fluctuation at the end, where the `ucss` is more stable.

These differences are due to the different regularization strategies of both algorithms:
* `us` regularizes the number of knots:
  * (+) it leads to long stretches of smooth approximations
  * (-) the ends become very instable
* `ucss` regularizes the weight of different basis functions, while keeping the same number of knots
  * (+) ends are very stable
  * (-) wavy derivatives due to the many knots
  
  
### Crossvalidation of Smoothing Parameters
We have implemented a normalization of the algorithms smoothing parameters, such that they only have to be chosen in the interval $[0,1]$. This way, we can visualize the crossvalidation error against the smoothing parameter to understand how it can be chosen automatically.

In [6]:
def crossvalidation_with_noise(filterset:str='BS3', noise:float=0.0):
    t, y = bldata[filterset].get_timeseries('C04')

    y = numpy.random.normal(y, scale=(y.max()-y.min())*noise)

    S = numpy.exp(numpy.linspace(-6, 0, 100))
    R_us = [
        bletl_analysis.splines._evaluate_smoothing_factor([s], t, y, k=10, method='us')
        for s in S
    ]
    R_ucss = [
        bletl_analysis.splines._evaluate_smoothing_factor([s], t, y, k=10, method='ucss')
        for s in S
    ]
    
    opt_us = S[numpy.argmin(R_us)]
    opt_ucss = S[numpy.argmin(R_ucss)]
    s_opt_us = bletl_analysis.splines._normalize_smoothing_factor('us', opt_us, t, y)
    s_opt_ucss = bletl_analysis.splines._normalize_smoothing_factor('ucss', opt_ucss, t, y)


    # plot residual vs. normalized smoothing factor
    fig, (left, right) = pyplot.subplots(ncols=2, figsize=(12,6), dpi=120)

    left.plot(S, R_us, label='us', color='orange')
    left.plot(S, R_ucss, label='ucss', color='blue')

    left.axvline(S[numpy.argmin(R_us)], label=f'opt us ({opt_us:.4f})', color='orange', linestyle='--')
    left.axvline(S[numpy.argmin(R_ucss)], label=f'opt ucss ({opt_ucss:.4f})', color='blue', linestyle='--')
    left.set_yscale('log')
    left.legend()

    left.set_xlabel('normalized smoothing factor')
    left.set_ylabel('10-fold crossvalidation error')
    left.set_title(f'filterset={filterset}, noise={noise*100:.0f} %')
    left.set_xscale('log')
    
    s_us = scipy.interpolate.UnivariateSpline(t, y, s=s_opt_us)
    s_ucss = bletl_analysis.UnivariateCubicSmoothingSpline(t, y, smooth=s_opt_ucss)

    x = numpy.linspace(0,20,1000)
    right.scatter(t, y, s=15, color='gray')
    right.plot(x, s_us(x), color='orange')
    right.plot(x, s_ucss(x), color='blue')
    right.set_xlabel('time   [h]')
    right.set_ylabel(filterset)
    right.set_title(f'fit with crossvalidated smoothing factor')

    pyplot.show()
    
ipywidgets.interact_manual(
    crossvalidation_with_noise,
    filterset=list(bldata.keys()),
    noise=ipywidgets.FloatSlider(min=0, max=0.1, step=0.01)
);

interactive(children=(Dropdown(description='filterset', options=('BS3', 'pH', 'DO'), value='BS3'), FloatSlider…

By increasing the relative noise in the interactive plot above, you can see how the dependency of the crossvalidation error on the smoothing factor becomes more convex.

One interpretation of this is, that when there's almost no noise, the linear interpolation is locally the best interpolation. Unfortunately, this goes hand in hand with unstable derivatives at the end of the approximation.

In the next plot, you can explore how the crossvalidation outperforms the conservatively set upper bound.

In [8]:
def test_exponential(noise=0):
    mue = 0.352
    y0 = 0.05
    t = numpy.linspace(0, 20, 50)
    y = y0 * numpy.exp(mue*t)
    y = numpy.random.normal(y, scale=noise*numpy.max(y))

    spline_us = bletl_analysis.get_crossvalidated_spline(t, y, method='us')
    spline_ucss = bletl_analysis.get_crossvalidated_spline(t, y, method='ucss')

    s = bletl_analysis.splines._normalize_smoothing_factor('us', 1, t, y)
    manual = scipy.interpolate.UnivariateSpline(t, y, s=s)

    actual_sse = numpy.square(manual(t) - y).sum()

    print(f's={s} resulted in sse={actual_sse}')
    
    fig, (left, right) = pyplot.subplots(ncols=2, figsize=(14,6), dpi=120)
    
    left.scatter(t, y)
    x = numpy.linspace(t[1], 20, 1000)
    
    for spline, name, color in zip(
            [spline_us, spline_ucss, manual],
            ['US+CV', 'UCSS+CV', 'US default'],
            ['orange', 'blue', 'green']
        ):
        left.plot(x, spline(x), label=name, color=color)
        growthrate = spline.derivative(1)(x) / spline(x)
        right.plot(x, growthrate, color=color, label=f'{name}: {numpy.median(growthrate):.3f}')
    
    left.set_xlabel('time')
    left.set_ylabel('value')
    left.legend()
    right.set_xlabel('time')
    right.set_ylabel('specific growth rate')
    right.set_ylim(-10,10)
    right.legend()
    
    pyplot.show()
    
ipywidgets.interact_manual(
    test_exponential,
    noise=ipywidgets.FloatSlider(min=0, max=0.1, step=0.01)
);

interactive(children=(FloatSlider(value=0.0, description='noise', max=0.1, step=0.01), Button(description='Run…